# Advanced SQL

## SQL Window Functions

### OVER and PARTITION

From <a href="https://www.postgresql.org/docs/9.1/tutorial-window.html">PostreSQL tutorial</a>:
<quote><p>"A window function performs a calculation across a set of table rows that are somehow related to the current row. This is comparable to the type of calculation that can be done with an aggregate function. But unlike regular aggregate functions, use of a window function does not cause rows to become grouped into a single output row — the rows retain their separate identities. Behind the scenes, the window function is able to access more than just the current row of the query result.</p>

<p>Here is an example that shows how to compare each employee's salary with the average salary in his or her department:"</p></quote>

<code>SELECT depname, empno, salary, avg(salary) OVER (PARTITION BY depname) FROM empsalary;</code>

As a results for each employee (empno) in addition to his individual salary we will have the average salary for his department (depname) in the column empsalary. 

**You can’t use window functions and standard aggregations in the same query. More specifically, you can’t include window functions in a GROUP BY clause.**

##### Example 1

The following code calculates sum for each day so that we can compare standard amount per one order to the amount per the entire day.  

<code>SELECT standard_amt_usd,
       SUM(standard_amt_usd) OVER (ORDER BY occurred_at) AS running_total
FROM orders</code>

##### Example 2

Calculate total amounts for each year:

<code>SELECT standard_amt_usd,
       DATE_TRUNC('year', occurred_at) as year,
       SUM(standard_amt_usd) OVER (PARTITION BY DATE_TRUNC('year', occurred_at) ORDER BY occurred_at) AS running_total
FROM orders
</code>

### RANK() and ROW_NUMBER()

From <a href="https://stackoverflow.com/questions/7747327/sql-rank-versus-row-number">StackOverflow</a>  

<b>ROW_NUMBER</b> : Returns a unique number for each row starting with 1. For rows that have duplicate values,numbers are arbitarily assigned.

<b>Rank</b> : Assigns a unique number for each row starting with 1,except for rows that have duplicate values,in which case the same ranking is assigned and a gap appears in the sequence for each duplicate ranking.

##### Example 1
The code does the following: 
for each account we order by total and assign rank according to that. So for each account-total pair their will be a ranking according to the value in the 'total' column. This ranking will be saved in total_rank column.

<code>SELECT id,
       account_id,
       total,
       RANK() OVER (PARTITION BY account_id ORDER BY total DESC) AS total_rank
FROM orders
</code>

### ALIASES

##### Example

<code>SELECT id,
       account_id,
       DATE_TRUNC('year',occurred_at) AS year,
       DENSE_RANK() OVER account_year_window AS dense_rank,
       total_amt_usd,
       SUM(total_amt_usd) OVER account_year_window AS sum_total_amt_usd,
       COUNT(total_amt_usd) OVER account_year_window AS count_total_amt_usd,
       AVG(total_amt_usd) OVER account_year_window AS avg_total_amt_usd,
       MIN(total_amt_usd) OVER account_year_window AS min_total_amt_usd,
       MAX(total_amt_usd) OVER account_year_window AS max_total_amt_usd
FROM orders
WINDOW account_year_window as (PARTITION BY account_id ORDER BY DATE_TRUNC('year',occurred_at))
</code>

### LAG and LEAD

LAG pulls from previous rows, LEAD pulls from following rows. Simply put, it allows to shift the column down or up so that we can compare them. 

<code> SELECT occurred_at,
       total_amt_usd,
       LEAD(total_amt_usd) OVER (ORDER BY occurred_at) AS lead,
       LEAD(total_amt_usd) OVER (ORDER BY occurred_at) - total_amt_usd AS lead_difference
FROM (
SELECT occurred_at,
       SUM(total_amt_usd) AS total_amt_usd
  FROM orders 
 GROUP BY 1
) sub
</code>

### QUANTILE and PERCENTILE   
The syntax is NTILE(*# of buckets*). In this case, ORDER BY determines which column to use to determine the quartiles (or whatever number of ‘tiles you specify). Number of buckets is an "integer ranging from 1 to the argument value, dividing the partition as equally as possible".

<code>SELECT account_id, occurred_at,
standard_qty,
NTILE(4) OVER (PARTITION BY account_id ORDER BY standard_qty) as quartile
FROM orders
</code>

<code>SELECT account_id, occurred_at,
standard_qty,
NTILE(2) OVER (PARTITION BY account_id ORDER BY gloss_qty) as gloss_half
FROM orders</code>

## Advanced JOINS

### Symmetric Difference

To get symmetic difference - records that are present only in one of the tables - either in A or in B. 

<code>SELECT column_name(s)
FROM Table_A
FULL OUTER JOIN Table_B ON Table_A.column_name = Table_B.column_name;    
WHERE Table_A.column_name IS NULL OR Table_B.column_name IS NULL
</code>

### Self JOINS

For each order in table o1 we find the orders that happend in that account AFTER that order. We also limit the orders to 28-day interval: only orders that wer made no later than 28 days after the order in table o1.

<code>SELECT o1.id AS o1_id,
       o1.account_id AS o1_account_id,
       o1.occurred_at AS o1_occurred_at,
       o2.id AS o2_id,
       o2.account_id AS o2_account_id,
       o2.occurred_at AS o2_occurred_at
  FROM orders o1
 LEFT JOIN orders o2
   ON o1.account_id = o2.account_id
  AND o2.occurred_at > o1.occurred_at
  AND o2.occurred_at <= o1.occurred_at + INTERVAL '28 days'
ORDER BY o1.account_id, o1.occurred_at
</code>                                     

### UNION

UNION removes duplicates. If we want to retain them, we should use UNION ALL.

## Queary Optimization

Things that affect speed: 
- Table size  
- Joins  
- Aggregations  

And also (but these are not controllable):  
- Other users running queries concurrently on the database  
- Database software and optimization (e.g., Postgres is optimized differently than Redshift)  


Tips to improve performance: 
- Filter the data set
- Write and test query on a subset 
- Aggregations happen before LIMIT, so LIMIT doesn't help
- For aggregations: do filtering in a subquery, and only then do the aggregation
- For testing query, reduce the size of a subset in a subquery
- Reduce table sizes before joining them
- Do aggregation in a subquery and then do the join. This way we will reduce the cost of join (since it will be performed on a smaller dataset. WARNING: do check the logic!!! 
- use EXPLAIN before the query to check the execution plan for the query 